In [0]:
GTFS_URL = "https://www.sptrans.com.br/umbraco/Surface/PerfilDesenvolvedor/BaixarGTFS?memberName=sptrans"
GTFS_PATH = "/Volumes/workspace/sptrans/bronze/gtfs"
TEMP_LOCAL = "/tmp/temp_gtfs"

import requests
import zipfile
import io
import pandas as pd
from pyspark.sql.functions import current_date
import os

# Download e extração
r = requests.get(GTFS_URL, verify=False)  # alguns servidores exigem verify=False
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(TEMP_LOCAL)

files = [
    "agency.txt", "routes.txt", "trips.txt", "stops.txt",
    "stop_times.txt", "shapes.txt", "calendar.txt", "calendar_dates.txt",
    "frequencies.txt", "fare_attributes.txt", "fare_rules.txt"
]

for f in files:
    local_path = f"{TEMP_LOCAL}/{f}"
    if os.path.exists(local_path):
        pdf = pd.read_csv(local_path)
        df = spark.createDataFrame(pdf).withColumn("load_date", current_date())

        df.write.format("delta") \
            .mode("overwrite") \
            .saveAsTable(f"workspace.sptrans.bronze_gtfs_{f.replace('.txt','')}")
    else:
        print(f"⚠ Não encontrado no ZIP: {local_path}")

print("✅ GTFS atualizado carregado com sucesso!")

/databricks/python/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sptrans.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


⚠ Não encontrado no ZIP: /tmp/temp_gtfs/calendar_dates.txt
✅ GTFS atualizado carregado com sucesso!
